In [ ]:
from pathlib import Path

import shapefile
import shapely.geometry as sgeom
from shapely.ops import unary_union

from amap import (
    polygon_center,
    shp_to_df,
    polygon_to_polys,
    shorten_ct_name,
    shorten_pr_name,
    make_prj_file,
)

In [ ]:
data_dirpath = Path('../data')

dt_shp_filepath = data_dirpath / 'cn_district.shp'
dt_csv_filepath = data_dirpath / 'cn_district.csv'

ct_shp_filepath = data_dirpath / 'cn_city.shp'
ct_prj_filepath = data_dirpath / 'cn_city.prj'
ct_csv_filepath = data_dirpath / 'cn_city.csv'

pr_shp_filepath = data_dirpath / 'cn_province.shp'
pr_prj_filepath = data_dirpath / 'cn_province.prj'
pr_csv_filepath = data_dirpath / 'cn_province.csv'

bd_shp_filepath = data_dirpath / 'cn_border.shp'
bd_prj_filepath = data_dirpath / 'cn_border.prj'

In [ ]:
dt_df = shp_to_df(dt_shp_filepath)
dt_df.to_csv(dt_csv_filepath, index=False)
dt_df.head(10)

区县多边形合成市级多边形，制作配套的属性表。

In [ ]:
with shapefile.Reader(dt_shp_filepath) as reader:
    shapeRecs = reader.shapeRecords()

with shapefile.Writer(ct_shp_filepath, shapeType=5) as writer:
    writer.fields = [
        ['pr_name', 'C', 80, 0],
        ['pr_adcode', 'N', 6, 0],
        ['ct_name', 'C', 80, 0],
        ['ct_adcode', 'N', 6, 0],
        ['short_name', 'C', 80, 0],
        ['lon', 'N', 7, 3],
        ['lat', 'N', 7, 3],
    ]
    for record in (
        dt_df.iloc[:, :4]
        .drop_duplicates('ct_name')
        .reset_index(drop=True)
        .to_dict(orient='records')
    ):
        polygons = []
        for shapeRec in shapeRecs:
            if shapeRec.record.ct_name == record['ct_name']:
                polygons.append(sgeom.shape(shapeRec.shape))
        polygon = unary_union(polygons)
        polys = polygon_to_polys(polygon)

        record['short_name'] = shorten_ct_name(record['ct_name'])
        record['lon'], record['lat'] = polygon_center(polygon)

        writer.record(**record)
        writer.poly(polys)

make_prj_file(ct_prj_filepath)
ct_df = shp_to_df(ct_shp_filepath)
ct_df.to_csv(ct_csv_filepath, index=False)
ct_df.head(10)

市级多边形合成省级多边形，制作配套的属性表。

In [ ]:
with shapefile.Reader(ct_shp_filepath) as reader:
    shapeRecs = reader.shapeRecords()

with shapefile.Writer(pr_shp_filepath, shapeType=5) as writer:
    writer.fields = [
        ['pr_name', 'C', 80, 0],
        ['pr_adcode', 'N', 6, 0],
        ['short_name', 'C', 80, 0],
        ['lon', 'N', 7, 3],
        ['lat', 'N', 7, 3],
    ]
    for record in (
        ct_df.iloc[:, :2]
        .drop_duplicates('pr_name')
        .reset_index(drop=True)
        .to_dict(orient='records')
    ):
        polygons = []
        for shapeRec in shapeRecs:
            if shapeRec.record.pr_name == record['pr_name']:
                polygons.append(sgeom.shape(shapeRec.shape))
        polygon = unary_union(polygons)
        polys = polygon_to_polys(polygon)

        record['short_name'] = shorten_pr_name(record['pr_name'])
        record['lon'], record['lat'] = polygon_center(polygon)

        writer.record(**record)
        writer.poly(polys)

make_prj_file(pr_prj_filepath)
province_df = shp_to_df(pr_shp_filepath)
province_df.to_csv(pr_csv_filepath, index=False)
province_df.head(10)

省级多边形合成国界多边形。

In [ ]:
with shapefile.Reader(pr_shp_filepath) as reader:
    polygons = list(map(sgeom.shape, reader.shapes()))
polygon = unary_union(polygons)
polys = polygon_to_polys(polygon)

make_prj_file(bd_prj_filepath)
with shapefile.Writer(bd_shp_filepath, shapeType=5) as writer:
    writer.fields = [['cn_name', 'C', 80, 0], ['cn_adcode', 'N', 6, 0]]
    writer.record(country_name='中华人民共和国', country_adcode=100000)
    writer.poly(polys)